<a href="https://colab.research.google.com/github/hargurjeet/MachineLearning/blob/master/ML_using_Kfolds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dataset from Kaggle

In [1]:
!pip install opendatasets --upgrade --quiet

In [14]:
import numpy as np
import pandas as pd
import opendatasets as od
from sklearn import model_selection

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
od.download('https://www.kaggle.com/c/30-days-of-ml/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: hargurjeet
Your Kaggle Key: ··········


100%|██████████| 66.0M/66.0M [00:00<00:00, 116MB/s]



Extracting archive ./30-days-of-ml/30-days-of-ml.zip to ./30-days-of-ml


# Splitting the data to kfold

In [4]:
train_fold = pd.read_csv("/content/30-days-of-ml/train.csv")

test = pd.read_csv("/content/30-days-of-ml/test.csv", index_col=0)

In [5]:
train_fold["kfold"] = -1

In [7]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=train_fold)):
    train_fold.loc[valid_indicies, "kfold"] = fold

In [8]:
train_fold.sample(5)

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
208250,347303,B,B,B,C,B,B,A,E,C,H,0.393696,0.562602,0.522218,0.520174,0.535155,0.362196,0.177250,0.368204,0.387330,0.550739,0.165794,0.301234,0.448767,0.773407,7.264243,4
210259,350691,A,B,A,C,B,B,A,E,C,A,0.544822,-0.053964,0.325364,0.363256,0.518960,0.397304,0.425360,0.345051,0.300312,0.357744,0.078063,0.283843,0.392208,0.475412,6.942858,2
239472,399345,A,A,A,C,B,D,A,E,A,F,0.743977,0.259932,0.954038,0.719666,0.285062,0.583169,0.436949,0.864470,0.924444,0.856345,0.687544,0.498958,0.782183,0.788264,7.790929,4
285289,475509,B,B,B,C,B,B,A,E,E,O,0.082274,0.350819,0.422431,0.312285,0.530356,0.454539,0.677573,0.318044,0.373369,0.279686,0.090899,0.423850,0.567382,0.474096,8.259257,1
72601,120813,B,B,A,C,B,B,A,E,C,F,0.500508,0.354153,0.543730,0.252242,0.286714,0.375897,0.353673,0.311817,0.315228,0.399464,0.368156,0.170242,0.554146,0.367345,9.105533,2


In [9]:
train_fold.kfold.value_counts()

4    60000
3    60000
2    60000
1    60000
0    60000
Name: kfold, dtype: int64

In [ ]:
train_fold.to_csv("train_folds.csv", index=False)

# Implementing Kfold on Random forests

In [12]:
numeric_cols

['id',
 'cont0',
 'cont1',
 'cont2',
 'cont3',
 'cont4',
 'cont5',
 'cont6',
 'cont7',
 'cont8',
 'cont9',
 'cont10',
 'cont11',
 'cont12',
 'cont13',
 'target',
 'kfold']

In [ ]:
useful_features = [cname for cname in train_fold.columns if cname not in ['id','target','kfold' ]]
numeric_cols = [cname for cname in train_fold.columns if train_fold[cname].dtype in ['int64', 'float64'] and cname not in ['id','target','kfold' ]]
object_cols = [cname for cname in train_fold.columns if train_fold[cname].dtype in ['object']]

final_predictions = []
scores = []
for fold in range(5):
    xtrain = train_fold[train_fold.kfold != fold].reset_index(drop=True)
    xvalid = train_fold[train_fold.kfold == fold].reset_index(drop=True)
    xtest = test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    scaler = StandardScaler()
    xtrain[numeric_cols] = scaler.fit_transform(xtrain[numeric_cols])
    xvalid[numeric_cols] = scaler.transform(xvalid[numeric_cols])
    xtest[numeric_cols] = scaler.transform(xtest[numeric_cols])

    model = RandomForestRegressor(n_estimators=10, random_state=42)
    # model = XGBRegressor(random_state=fold)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))

0 0.770165644079102
1 0.7693716515494292
2 0.7718354497330934


In [ ]:
final_predictions

In [ ]:
predictions = np.mean(np.column_stack(final_predictions), axis=1)